<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [23]:
# TODO - Words, words, mere words, no matter from the heart.

from urllib.request import urlopen

res = urlopen("https://www.gutenberg.org/files/100/100-0.txt")
if res.status == 200:
    text = res.read().decode("utf-8")

In [24]:
import re

text = re.sub(r"([\\.].)", " ", text)
text = re.sub(r"[^A-Za-z0-9 ]", "", text)

In [60]:
chars = list(set(text))

i_to_c = {i: c for i, c in enumerate(chars)}
c_to_i = {c: i for i, c in enumerate(chars)}

encoded = [c_to_i[c] for c in text]
sequences = []
next_char = []

In [61]:
maxlen = 30
step = 5

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])

    

In [62]:
encoded

[26,
 62,
 17,
 24,
 34,
 0,
 21,
 38,
 36,
 48,
 21,
 34,
 29,
 2,
 34,
 62,
 37,
 56,
 38,
 4,
 20,
 34,
 38,
 41,
 17,
 19,
 12,
 18,
 34,
 21,
 34,
 38,
 57,
 17,
 62,
 22,
 56,
 38,
 17,
 23,
 38,
 57,
 3,
 18,
 18,
 3,
 54,
 19,
 38,
 25,
 20,
 54,
 22,
 34,
 56,
 12,
 34,
 54,
 62,
 34,
 38,
 2,
 27,
 38,
 57,
 3,
 18,
 18,
 3,
 54,
 19,
 25,
 20,
 54,
 22,
 34,
 56,
 12,
 34,
 54,
 62,
 34,
 4,
 20,
 3,
 56,
 38,
 34,
 49,
 17,
 17,
 22,
 38,
 3,
 56,
 38,
 23,
 17,
 62,
 38,
 21,
 20,
 34,
 38,
 48,
 56,
 34,
 38,
 17,
 23,
 38,
 54,
 29,
 27,
 17,
 29,
 34,
 38,
 54,
 29,
 27,
 33,
 20,
 34,
 62,
 34,
 38,
 3,
 29,
 38,
 21,
 20,
 34,
 38,
 5,
 29,
 3,
 21,
 34,
 59,
 38,
 25,
 21,
 54,
 21,
 34,
 56,
 38,
 54,
 29,
 59,
 19,
 17,
 56,
 21,
 38,
 17,
 21,
 20,
 34,
 62,
 38,
 12,
 54,
 62,
 21,
 56,
 38,
 17,
 23,
 38,
 21,
 20,
 34,
 38,
 33,
 17,
 62,
 18,
 59,
 38,
 54,
 21,
 38,
 29,
 17,
 38,
 0,
 17,
 56,
 21,
 38,
 54,
 29,
 59,
 38,
 33,
 3,
 21,
 20,
 38,
 54,
 18,
 

In [63]:
import numpy as np

X = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, seq in enumerate(sequences):
    for t, char in enumerate(seq):
        X[i, t, char] = 1
    y[i, next_char[i]] = 1

X.shape

(1037905, 30, 63)

In [64]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

max_features = 20000

model = Sequential()
model.add(LSTM(len(chars), input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam")

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 63)                32004     
_________________________________________________________________
dense_8 (Dense)              (None, 63)                4032      
Total params: 36,036
Trainable params: 36,036
Non-trainable params: 0
_________________________________________________________________


In [65]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, c_to_i[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = i_to_c[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
import random, sys

model.fit(X, y, batch_size=128, epochs=10, callbacks=[print_callback],)

Train on 1037905 samples
Epoch 1/10
1037824/1037905 [============================>.] - ETA: 0s - loss: 2.2824
----- Generating text after Epoch: 0
----- Generating with seed: "ll never hold that man my frie"
ll never hold that man my fries  JUIVANGUSS Orothim of to fors hish What dutsons you thy some for thourd a on andies far scoode    wirguod thim     Bure bot be fif goe noY hil shouny teres    ITht ang cnakice dringRangure is eriss deow he home lives and gere coth or hat      ang somandOO Castary Thas Fomest yiu gureeOThat the of bodest thig baghy the afe he of feencbeld anvor wemaruing    Wiylat be thie Voll dle gigeende thou 
1037905/1037905 [==============================] - 313s 302us/sample - loss: 2.2824
Epoch 2/10
1037824/1037905 [============================>.] - ETA: 0s - loss: 2.0014
----- Generating text after Epoch: 1
----- Generating with seed: "d room enoughWhen there is in "
d room enoughWhen there is in mansirk wrange    Andreon Bensade weld lasse SDOUK These workned

In [45]:
model = Sequential()

model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

model.summary()

unicorns = model.fit(
    x_train, y_train,
    batch_size=batch_size, 
    epochs=5,
    callbacks=[print_callback],
)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         2560000   
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


NameError: name 'x_train' is not defined

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN